In [4]:
import pandas as pd
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt

### Read org data

In [5]:
diff_pca = pd.read_csv("../../data/pca_5c.csv")
diff_pca.shape

(5148, 6)

In [6]:
diff_pca

,Date,pca_0,pca_1,pca_2,pca_3,pca_4
0,2023-04-12,1.909963,-0.351057,-0.056936,-0.028624,-0.040396
1,2023-04-11,1.717435,-0.471007,-0.047943,-0.000686,-0.015010
2,2023-04-10,1.609087,-0.467092,-0.055264,0.009463,-0.022080
3,2023-04-06,1.729451,-0.458372,-0.062140,0.008793,-0.004786
4,2023-04-05,1.635978,-0.502687,-0.074070,0.019104,-0.003345
...,...,...,...,...,...,...
5143,2003-04-17,0.367865,-0.074382,-0.085642,-0.008546,-0.001766
5144,2003-04-16,0.242211,-0.119124,-0.087902,0.008568,0.000521
5145,2003-04-15,0.265390,-0.092445,-0.045351,0.055036,-0.000707
5146,2003-04-14,0.273054,-0.011274,0.033414,0.059628,-0.007909


In [3]:
df_agg = pd.read_parquet("../../data/data_agg_v0.parquet")

price_cols = [x for x in df_agg.columns if "LAST" in x]
df_subset = df_agg[["Date"] + price_cols[:12]]
df_subset.index = df_subset["Date"]
df_subset = df_subset.drop(columns=["Date"])
print(f"Shape before dropping nulls, {df_subset.shape}")
df_subset = df_subset.dropna()
print(f"Shape after dropping nulls, {df_subset.shape}")

Shape before dropping nulls, (5152, 12)
Shape after dropping nulls, (5148, 12)


In [ ]:
df_subset1 = df_subset.sort_index()

In [ ]:
mask1 = df_subset1["PX_LAST_6"]>df_subset1["PX_LAST"]
mask2 = df_subset1["PX_LAST_12"]>df_subset1["PX_LAST_6"]

df_subset1["contango"] = (mask1 & mask2).astype("int")
df_subset1["contango"].mean()

In [ ]:
contango = df_subset1[["contango"]]
for i in range(1,5):
    contango[f"lag_{i}"] = contango["contango"].shift(i)
contango["contango_start"] = ((contango.sum(axis=1)==1) & (contango["contango"]==1)).astype(int)

In [ ]:
contango["contango_start"].mean()

In [ ]:
fig,ax = plt.subplots(figsize=(16,9))
df_subset1[df_subset1.contango>0].drop(columns="contango").groupby(lambda x: (x.year)).mean().T.plot(ax=ax)

In [ ]:
pca_data = pd.read_csv("../../data/log_returns_pca.csv",index_col=0)
pca_data.index = pd.to_datetime(pca_data.index)
pca_data.shape

In [ ]:
df_merged = df_subset1[["contango"]].merge(pca_data,left_index=True,right_index=True)
df_merged[[x for x in df_merged.columns if "pca" in x\
          ]] = df_merged[[x for x in df_merged.columns if "pca" in x]].shift(1)

In [ ]:
df_merged1 = df_merged.dropna()

import statsmodels.api as sm
Xtrain = df_merged1.values
y = df_merged1["contango"]

# building the model and fitting the data
log_reg = sm.Logit(y, Xtrain).fit()

In [ ]:
Xtrain.shape

In [ ]:
print(log_reg.summary())